In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd.functional import jacobian
from functorch import vmap, jacrev
import matplotlib.pyplot as plt
import datetime

In [2]:
# ------------------------------
# Set up the device: GPU if available, otherwise CPU
# ------------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# ------------------------------
# Define ODE constants and the ODE function φ on the selected device
# ------------------------------
D = torch.tensor([-9.54, -8.16, -4.26, -11.43], dtype=torch.float32, device=device)
A = torch.tensor(
    [[3.18, 2.72, 1.42, 3.81]], dtype=torch.float32, device=device
)  # shape: (4,)
b = torch.tensor([[7.81]], dtype=torch.float32, device=device)  # scalar


# def phi(y):
#     """
#     Given y (a tensor of shape (5,)), compute:
#       x = y[:4]
#       u = y[4:5]
#       m = max(0, u + dot(A, x) - b)
#     and return
#       [ -(D + A*m); m - u ] as a tensor of shape (5,)
#     """
#     x = y[:4]
#     u = y[4]
#     m = torch.clamp(u + torch.dot(A, x) - b, min=0.0)
#     top = -(D + A * m)
#     bottom = m - u
#     return torch.cat((top, bottom.unsqueeze(0)))  # resulting shape: (5,)


def createPhi(D, A, b):
    _, n = A.shape

    def phi(y):
        """
        Given y (a tensor of shape (6,)), where:
          x = y[:4] ∈ ℝ⁴,
          u = y[4:] ∈ ℝ² (treated as a column vector),
        compute:
          m = max(0, u + A @ x - b) ∈ ℝ²,
        and return:
          [ -(D + Aᵀ @ m);  m - u ] as a tensor of shape (6,).

        Note: This function assumes:
          - A is a tensor of shape (2,4)
          - D is a tensor of shape (4,)
          - b is a scalar.
        """
        # Convert x to a column vector of shape (4,1)
        x = y[:n]  # shape: (4,1)
        # Convert u to a column vector of shape (2,1)
        u = y[n:]  # shape: (2,1)

        # Compute m = max(0, u + A @ x - b)
        m = torch.clamp(u + A @ x - b, min=0.0)  # shape: (2,1)

        # Compute top = -(D + Aᵀ @ m)
        # Ensure D is used as a column vector by unsqueezing it.
        top = -(D.unsqueeze(1) + A.t() @ m)  # shape: (4,1)

        # Compute bottom = m - u
        bottom = m - u  # shape: (2,1)

        # Concatenate top and bottom along the first dimension and squeeze to get shape (6,)
        return torch.cat((top, bottom), dim=0).squeeze(1)

    return phi


phi = createPhi(D, A, b)


# ------------------------------
# Define the PINN model (moved to GPU)
# ------------------------------
class PINN(nn.Module):
    def __init__(self):
        super(PINN, self).__init__()
        # Two-layer network: input dimension 1 -> 100 -> 5
        self.fc1 = nn.Linear(1, 100)
        self.fc2 = nn.Linear(100, 5)
        self.activation = nn.Tanh()

    def forward(self, t):
        """
        Forward pass for a scalar (or batch) time input.
        We assume t is a tensor of shape (N, 1) or a scalar tensor.
        The network output is modulated as:
            ŷ(t) = (1 - exp(-t)) * NN(t)
        to enforce ŷ(0) = 0.
        """
        if t.dim() == 0:
            t = t.unsqueeze(0)
        x = self.activation(self.fc1(t))
        out = self.fc2(x)
        return (1 - torch.exp(-t)) * out


model = PINN().to(device)

Using device: cuda


In [ ]:
# Recreate the model architecture and move it to the appropriate device

# Get the current date in YYYY_MM_DD format
current_date = datetime.date.today().strftime("%Y_%m_%d")
filename = f"pinn_model_{current_date}.pt"


model_reloaded = PINN().to(device)


# Load the saved state dictionary


model_reloaded.load_state_dict(torch.load(filename, map_location=device))


# Optionally, set the model to evaluation mode
model_reloaded.eval()

PINN(
  (fc1): Linear(in_features=1, out_features=100, bias=True)
  (fc2): Linear(in_features=100, out_features=5, bias=True)
  (activation): Tanh()
)

In [6]:
# ------------------------------
# Evaluate the trained model at select time points
# ------------------------------
test_times = [0.0, 2.5, 5.0, 7.5, 10.0, 10.1]
for t in test_times:
    t_tensor = torch.tensor(t, dtype=torch.float32, device=device, requires_grad=True)
    y_pred = model_reloaded(t_tensor)
    print(f"t = {t:4.1f}, ŷ(t) = {y_pred.detach().cpu().numpy().flatten()}")

t =  0.0, ŷ(t) = [ 0.  0.  0.  0. -0.]
t =  2.5, ŷ(t) = [0.76027703 0.64399105 0.3438906  0.8967337  2.743333  ]
t =  5.0, ŷ(t) = [0.7262206  0.6388246  0.33385092 0.86810476 2.9816902 ]
t =  7.5, ŷ(t) = [0.72618747 0.6352146  0.33321396 0.8663009  2.999563  ]
t = 10.0, ŷ(t) = [0.73239946 0.62363756 0.33278432 0.86955106 2.9987428 ]
t = 10.1, ŷ(t) = [0.7327253  0.62304866 0.3327165  0.86973536 2.9986863 ]
